<a href="https://colab.research.google.com/github/WetSuiteLeiden/data-collection/blob/master/koop_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook

Using KOOP's SRU interface to Officiele Publicaties,
right now to fetch Parliamentary documents (Kamerstukken, Handelingen, Aanhangsels bij de Handelingen).

This is less to show that interface - which is pretty good - and more to fetch enough to make a dataset.

# Fetching

In [1]:
import collections, datetime, random, time, pprint

import wetsuite.helpers.notebook
import wetsuite.helpers.localdata
import wetsuite.datacollect.koop_sru 
import wetsuite.helpers.date
import wetsuite.helpers.etree
import wetsuite.helpers.koop_parse

In [ ]:
sru_op = wetsuite.datacollect.koop_sru.OfficielePublicaties()

sru_op.explain_parsed()

In [2]:
# store to put downloads into:
op_fetched = wetsuite.helpers.localdata.LocalKV( 'op_fetched.db', str, bytes )

In [ ]:
#sru_op.search_retrieve_many( 'dt.language = nl', at_a_time=500, up_to=50000, callback=op_callback)
#sru_op.search_retrieve_many( 'w.dossiernummer = 26100', at_a_time=50, up_to=1000, callback=op_callback)
#sru_op.search_retrieve( 'w.publicatienaam = Kamerstuk', callback=op_callback)
#query = 'w.publicatienaam = Kamerstuk  AND  w.ondernummer > 0 AND  w.documentstatus = Opgemaakt'
#sru_op.search_retrieve(       'w.publicatienaam = Kamerstuk  AND  w.documentstatus = Opgemaakt', callback=op_callback)

In [ ]:
count_pubnaam = collections.defaultdict( int )
count_status  = collections.defaultdict( int )


#query = 'dt.identifier any kst-'
#query = 'dt.identifier any h-'
#query = 'dt.identifier any ah-'
query = 'dt.identifier any ah- OR dt.identifier any h- OR dt.identifier any kst-'
#query = 'dt.language = nl'
sru_op.search_retrieve( query )

pbar = wetsuite.helpers.notebook.progress_bar( sru_op.num_records(), description='fetching' )

#metas = []

def op_callback( record_node, verbose=0 ):
    ''' Read search result records, pick out the URLs to fetch and fetch them. 
        Is a local function because we count per query, in a slightly weirdly scoped way '''
    pbar.value       += 1
    
    #if verbose:
    #    print( wetsuite.helpers.etree.debug_pretty( record_node ) )

    meta = wetsuite.helpers.koop_parse.parse_op_searchmeta( record_node, flatten=True )
    if verbose:
        pprint.pprint( meta )
    #pprint.pprint( meta['publicatienaam'] )
    #metas.append( meta )
    #count_pubnaam[ meta['publicatienaam'] ] += 1
    #return

    manifs = meta['manifestations'] 
    manifkeys = list(manifs.keys())

    chosen_keys = wetsuite.helpers.koop_parse.prefer_types(
        manifkeys,
        all_of=  ('metadata',),
        first_of=('xml','html', 'odt','pdf'),
        never=   ('metadataowms'),
        require_present=(),
        )
    #print( '%r -> %r'%( manifkeys, chosen_keys ) )

    for chosen_key in chosen_keys:
        chosen_url = manifs[chosen_key]
        #print( chosen_url )

        try:
            _, came_from_cache = wetsuite.helpers.localdata.cached_fetch( op_fetched, chosen_url )
            if not came_from_cache:
                count_status['fetched'] += 1
                #time.sleep( 2 ) # be somewhat nice to the servers
                time.sleep( 0.1 ) # be somewhat nice to the servers
            else:
                count_status['cached'] += 1
        # mainly expecting 404, 500
        except ValueError as exc:
            count_status['error'] += 1
            print( "ERROR downloading %s: %s  for %r"%(chosen_key, exc, chosen_url))
            time.sleep( 10 ) # be somewhat nicer to the servers

    pbar.description  = ', '.join( list( '%s:%d'%(k, v)  for k,v in count_status.items()) )


try:
    sru_op.search_retrieve_many( query, at_a_time=500, up_to=5000000, callback=op_callback)
except ValueError as e:
    print( "ERROR: %s"%(e) )
    raise

# Forming into a dataset

So, we now have plenty of data to work with:

In [3]:
op_fetched.summary(True)

{'size_bytes': 16403304448,
 'size_readable': '15GiB',
 'num_items': 955902,
 'avgsize_bytes': 17160,
 'avgsize_readable': '17KiB'}

the largest detail is that for each conceptual item (identifier), we have at least a small metadata file and a content file.

Also, we know that in our download code above we expressed a _preference_ for XML, but it may not always be.
So let's check through that.

First, group the metadata together with the content based on the identifier mentioned in the URL:

In [4]:
somekeys = list( op_fetched.keys() )
#somekeys = list( op_fetched.keys() )#[:10000]
#somekeys = list( op_fetched.random_keys(10000) )

In [6]:
docmeta_groups = collections.defaultdict(dict)
typecount = collections.defaultdict(int)
others = []

import re
# the regexp is partly to some basic check of our assumptions about the parts of those URLs for this subset -- seems prudent since we'll also be extracting information from it
_re_repourl_parl = re.compile( r'^https?://repository.overheid.nl/frbr/officielepublicaties/([a-z-]+)/([A-Za-z0-9-]+)/([A-Za-z0-9_-]+)/([a-z0-9-]+)/([a-z]+)/([A-Za-z0-9_-]+.[a-z]+)$' )
#                                                                        op       h-ek       20092010     h-ek-20092010-1_2       1       xml    h-ek-20092010-1_2.xml

for key in somekeys:
    #print( key )
    match = _re_repourl_parl.match(key)
    if match is None:
        print( f'ERROR for {repr(key)}' )
    else:
        doctype, group, id, mn, mtype, bn = match.groups()
        #print(id, mtype)
        docmeta_groups[id][mtype] = key
        typecount[mtype] += 1
        if mtype not in ('metadata','xml'): # 
            others.append( key )

    #print( match , key )

#groups
typecount

defaultdict(int, {'metadata': 480140, 'xml': 473483, 'pdf': 2276, 'odt': 3})

From memory, the office documents might be temporarily kept in that format while new, and eventually replaced with PDFs, 
and in any case they are rare enough to ignore.

I would need to double-check about the `onopgemaakt` documents within `blg` and `ah`, which is what almost all of those ~2200 content URLs are:

In [ ]:
others

In [33]:
op_selection = wetsuite.helpers.localdata.LocalKV( 'op_selection.db', str, bytes )
op_selection.truncate()

In [34]:
import wetsuite.helpers.format
import wetsuite.helpers.notebook

# store to put downloads into:
vergaderjaar_bytecount = collections.defaultdict(int)

for id in wetsuite.helpers.notebook.ProgressBar(docmeta_groups):
    if 'metadata' in docmeta_groups[id] and 'xml' in docmeta_groups[id]:
        meta_url = docmeta_groups[id]['metadata'] # implicityly a test that we _have_ metadata for all
        meta_list = wetsuite.helpers.koop_parse.parse_op_metafile( op_fetched.get(meta_url) )
        #pprint.pprint( meta_dict )

        xml_url = docmeta_groups[id]['xml'] # implicityly a test that we _have_ metadata for all
        xml_bytes = op_fetched.get(xml_url)
        #print( wetsuite.helpers.etree.debug_pretty(xml_bytes) )# meta_dict = wetsuite.helpers.koop_parse.parse_op_searchmeta(  )

        for kk,tt,vv in meta_list:
            if kk=='vergaderjaar':
                vergaderjaar_bytecount[ vv ] += len(xml_bytes)
                if vv in (#'2020-2021', '2021-2022', 
                          '2022-2023', '2023-2024'):
                    op_selection.put(id, xml_bytes, commit=False)
                break

op_selection.commit()
        #break




  0%|          | 0/480140 [00:00<?, ?it/s]

In [31]:
# dict( vergaderjaren )
for key in sorted(vergaderjaar_bytecount):
    print(key, wetsuite.helpers.format.kmgtp( vergaderjaar_bytecount[key] ) )

1994-1995 142M
1995-1996 281M
1996-1997 306M
1997-1998 293M
1998-1999 300M
1999-2000 343M
2000-2001 350M
2001-2002 315M
2002-2003 289M
2003-2004 369M
2004-2005 352M
2005-2006 342M
2006-2007 270M
2007-2008 351M
2008-2009 399M
2009-2010 402M
2010-2011 457M
2011-2012 505M
2012-2013 471M
2013-2014 507M
2014-2015 544M
2015-2016 557M
2016-2017 464M
2017-2018 511M
2018-2019 570M
2019-2020 602M
2020-2021 664M
2021-2022 645M
2022-2023 690M
2023-2024 480M


In [16]:
count_pubnaam
#'Staatscourant': 44114,
#'Gemeenteblad': 3482,
#'Kamerstuk': 1950,
#'Niet-dossierstuk': 303,
#'Handelingen': 151})

defaultdict(int, {'Kamerstuk': 328877})

In [23]:
metas[0]

{'identifier': 'stcrt-2016-18738',
 'title': 'Openbaar exploot',
 'language': 'nl',
 'type': 'Gerechtelijke aankondigingen | Overige aankondigingen (OVERHEIDop.Staatscourant),  overige overheidsinformatie (OVERHEIDop.Rubriek)',
 'creator': 'Koninklijke Beroepsorganisatie van Gerechtsdeurwaarders (OVERHEID.OpenbaarLichaamVoorBedrijfEnBeroep)',
 'modified': '2016-05-08',
 'authority': 'Koninklijke Beroepsorganisatie van Gerechtsdeurwaarders (OVERHEID.OpenbaarLichaamVoorBedrijfEnBeroep)',
 'available': '2016-04-08',
 'date': '2016-04-08',
 'hasVersion': '1 (https://zoek.officielebekendmakingen.nl/stcrt-2016-18738.html)',
 'subject': 'Recht | Burgerlijk recht (OVERHEID.TaxonomieBeleidsagenda)',
 'publisher': 'Ministerie van Binnenlandse Zaken (OVERHEID.Ministerie)',
 'product-area': 'officielepublicaties',
 'content-area': 'officielepublicaties/stcrt/2016',
 'jaargang': '2016',
 'organisatietype': 'openbaar lichaam voor bedrijf en beroep (OVERHEID.Organisatietype)',
 'persoonsgegevens': 'A

In [24]:
import pandas
df = pandas.DataFrame(metas)

#import collections
#ollections.Counter(meta['jaargang']


In [29]:
metas[0]

{'identifier': 'stcrt-2016-18738',
 'title': 'Openbaar exploot',
 'language': 'nl',
 'type': 'Gerechtelijke aankondigingen | Overige aankondigingen (OVERHEIDop.Staatscourant),  overige overheidsinformatie (OVERHEIDop.Rubriek)',
 'creator': 'Koninklijke Beroepsorganisatie van Gerechtsdeurwaarders (OVERHEID.OpenbaarLichaamVoorBedrijfEnBeroep)',
 'modified': '2016-05-08',
 'authority': 'Koninklijke Beroepsorganisatie van Gerechtsdeurwaarders (OVERHEID.OpenbaarLichaamVoorBedrijfEnBeroep)',
 'available': '2016-04-08',
 'date': '2016-04-08',
 'hasVersion': '1 (https://zoek.officielebekendmakingen.nl/stcrt-2016-18738.html)',
 'subject': 'Recht | Burgerlijk recht (OVERHEID.TaxonomieBeleidsagenda)',
 'publisher': 'Ministerie van Binnenlandse Zaken (OVERHEID.Ministerie)',
 'product-area': 'officielepublicaties',
 'content-area': 'officielepublicaties/stcrt/2016',
 'jaargang': '2016',
 'organisatietype': 'openbaar lichaam voor bedrijf en beroep (OVERHEID.Organisatietype)',
 'persoonsgegevens': 'A

In [28]:
df

,identifier,title,language,type,creator,modified,authority,available,date,hasVersion,...,datumOndertekening,externeBijlage,datumTotstandkoming,plaatsTotstandkoming,sysyear,sysnumber,sysseqnumber,verdragnummer,eindpagina,startpagina
0,stcrt-2016-18738,Openbaar exploot,nl,Gerechtelijke aankondigingen | Overige aankond...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2016-05-08,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2016-04-08,2016-04-08,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,stcrt-2021-36478,gerechtelijke aankondiging | overige aankondiging,nl,Gerechtelijke aankondigingen | Overige aankond...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2021-08-20,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2021-07-21,2021-07-21,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stcrt-2019-5300,Oproepingen | Deurwaardersexploten,nl,Oproepingen | Deurwaardersexploten (OVERHEIDop...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2022-05-30,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2019-01-30,2019-01-30,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gmb-2021-426816,uitschrijving basisregistratie personen,nl,uitschrijving basisregistratie personen (OVERH...,Zwartewaterland (OVERHEID.Gemeente),2021-12-26,Zwartewaterland (OVERHEID.Gemeente),2021-11-26,2021-11-26,1 (https://zoek.officielebekendmakingen.nl/gmb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,stcrt-2016-72720,Openbaar exploot,nl,Gerechtelijke aankondigingen | Overige aankond...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2017-01-28,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2016-12-29,2016-12-29,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293995,stcrt-2020-16151,Gerechtelijke aankondigingen | Overige aankond...,nl,Gerechtelijke aankondigingen | Overige aankond...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2020-04-16,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2020-03-17,2020-03-17,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293996,stcrt-2022-4912,gerechtelijke aankondiging | overige aankondiging,nl,overige overheidsinformatie (OVERHEIDop.Rubriek),Koninklijke Beroepsorganisatie van Gerechtsdeu...,2022-03-16,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2022-02-14,2022-02-14,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293997,stcrt-2023-28326,gerechtelijke aankondiging | overige aankondiging,nl,overige overheidsinformatie (OVERHEIDop.Rubriek),Koninklijke Beroepsorganisatie van Gerechtsdeu...,2023-11-11,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2023-10-12,2023-10-12,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293998,stcrt-2018-31333,Openbaar exploot,nl,Gerechtelijke aankondigingen | Overige aankond...,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2018-07-04,Koninklijke Beroepsorganisatie van Gerechtsdeu...,2018-06-04,2018-06-04,1 (https://zoek.officielebekendmakingen.nl/stc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df['publicatienaam'].value_counts()

Staatscourant                  258717
Gemeenteblad                    20507
Kamerstuk                       11789
Niet-dossierstuk                 1812
Handelingen                      1140
Kamervragen zonder antwoord        29
Staatsblad                          4
Tractatenblad                       2
Name: publicatienaam, dtype: int64